In [ ]:
import requests
import tarfile
import numpy as np
import cv2
from io import BytesIO
from skimage import io
from skimage.util import montage
import matplotlib.pyplot as plt

def apply_logarithm(image):
    img_float = np.float32(image)
    log_image = np.log(1 + img_float)
    log_image = cv2.normalize(log_image, None, 0, 255, cv2.NORM_MINMAX)
    return np.uint8(log_image)

def apply_exponential(image):
    img_float = np.float32(image)
    img_float /= 255.0
    exp_image = np.exp(img_float) - 1
    exp_image = cv2.normalize(exp_image, None, 0, 255, cv2.NORM_MINMAX)
    return np.uint8(exp_image)

def apply_mean_filter(image, kernel_size=3):
    kernel = np.ones((kernel_size, kernel_size), np.float32) / (kernel_size ** 2)
    mean_filtered_image = cv2.filter2D(image, -1, kernel)
    return mean_filtered_image

def convert_to_rgb(image):
    # Converte de BGR para RGB se necessário
    if len(image.shape) == 3 and image.shape[2] == 3:  # Checa se é uma imagem colorida
        return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image  # Mantém a imagem como está se for escala de cinza

def display_images(images, title):
    # Converte para RGB antes de criar o montage
    rgb_images = [cv2.cvtColor(img[0], cv2.COLOR_BGR2RGB) if len(img[0].shape) == 3 else img[0] for img in sorted(images, key=lambda x: x[1])]
    montage_image = montage(rgb_images, grid_shape=(20, 4), padding_width=150, fill=(255, 255, 255), channel_axis=3)
    
    # Plot the montage
    fig, ax = plt.subplots(figsize=(112, 84))
    ax.set_title(title, fontsize=16)
    ax.imshow(montage_image)
    ax.axis('off')

    # Add the images' names above them
    img_height = montage_image.shape[0] // 20  # Image's height
    img_width = montage_image.shape[1] // 4   # Image's width

    # Add the images' names above them
    for i, (img, name) in enumerate(sorted(images, key=lambda x: x[1])):
        row = i // 4
        col = i % 4
        x_pos = col * (img_width) + (img_width // 2)  # Centralize horizontally
        y_pos = row * (img_height-7) + 100 # Adjust vertically

        ax.text(x_pos, y_pos, name, color='black', fontsize=12, ha='center', va='bottom')

    plt.show()

def process_images_from_tar_gz(url, fruitFolders):
    # Download the .tar.gz file in memory
    response = requests.get(url)
    response.raise_for_status()
    
    # Use BytesIO to treat the content as a file
    file_like_object = BytesIO(response.content)
    
    # Open the tar.gz file directly from the byte stream
    with tarfile.open(fileobj=file_like_object, mode='r:gz') as tar:
        # Get all members (files) from .tar.gz
        members = tar.getmembers()

        # Filter and sort the directories in alphabetical order
        directories = sorted([m for m in members if m.isdir()], key=lambda x: x.name)

        for directory in directories:
            fruit = directory.name.split('/')[-1]  # Get the fruit name from the directory path
            if fruit not in fruitFolders:
                continue  # Skip if the fruit is not in the list

            # Get image files in the directory
            images = sorted([m for m in members if m.name.startswith(directory.name) and m.isfile()], key=lambda x: x.name)

            # Read and process the images
            processed_images = []
            for img_member in images:
                file = tar.extractfile(img_member)
                if file:
                    # Read the image
                    image = io.imread(file)
                    if image is None:
                        continue  # Ignore if not an image

                    # Apply transformations
                    logImage = apply_logarithm(image)
                    expImage = apply_exponential(image)
                    meanFilteredImage = apply_mean_filter(image)

                    # Create names for transformed images
                    imageName = img_member.name.split('/')[-1]
                    logImageName = f"log_{imageName}"
                    expImageName = f"exp_{imageName}"
                    meanImageName = f"mean_{imageName}"

                    # Add the original and transformed images with their names to the list
                    processed_images.append((image, imageName))
                    processed_images.append((logImage, logImageName))
                    processed_images.append((expImage, expImageName))
                    processed_images.append((meanFilteredImage, meanImageName))

            # Display the images using montage
            display_images(processed_images[:80], title=f"{fruit.capitalize()}")

url = 'https://github.com/GabrielSMartinelli/Proj_PDI/raw/main/fruits-DL-1024x768.tar.gz'
fruitFolders = ['apple', 'araticum', 'avocado', 'banana', 'butia', 'khaki', 'mango', 'orange', 'star_fruit', 'ugli_fruit']
process_images_from_tar_gz(url, fruitFolders)